# データ分析用ノートブック

## ライブラリのインポート

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from lib import Util

## 設定ファイルの読み込み

In [ ]:
# 設定ファイルの読み込み
with open(f"{Util.get_root_dir()}/../config/config.json", "r") as f:
    config = json.load(f)

## グラフ描画クラス

In [ ]:
class Plotter:
    """プロットを行うクラス"""
    def __init__(self, df):
        self.df     = df
        self.figure = None

    def heatmap(self, figsize:tuple=(100, 100), cmap:str="coolwarm", linewidths:float=0.2):
        """ヒートマップを描画する"""
        self.figure = plt.figure(figsize=figsize)
        plt.title("Correlation Heatmap")
        sns.heatmap(self.df, cmap=cmap, linewidths=linewidths)

    def save_fig(self, path:str):
        """図を保存する"""
        plt.savefig(path)
        plt.close()

## サブキャリア間の相関分析

In [ ]:
# 共通ファイルを取得
common_file = Util.get_common_files(path_list=[f"{Util.get_root_dir()}/../data/preprocessed-data/{field_device}/amp/" for field_device in config["FieldDevice"]["Pcap"]])

### ヒートマップ

In [ ]:
# 各ファイルに対して相関係数のヒートマップを描画
for field_device in config["FieldDevice"]["Pcap"]:
    for file_name in tqdm(common_file):
        # ファイルの読み込み
        df = pd.read_csv(f"{Util.get_root_dir()}/../data/preprocessed-data/{field_device}/amp/{file_name}", index_col=0)
        # 相関係数のヒートマップを描画
        plotter = Plotter(df=df.corr())
        plotter.heatmap()
        Util.create_path(f"{Util.get_root_dir()}/documents/{field_device}/amp/")
        plotter.save_fig(f"{Util.get_root_dir()}/documents/{field_device}/amp/{Util.remove_extension(file_name)}.png")